In [0]:
import os
import sys

import tensorflow as tf
import random
import numpy as np
import pandas as p
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

# Eager Execution

TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. To follow along with this guide, run the code samples below in an interactive python interpreter.

Eager execution is a flexible machine learning platform for research and experimentation, providing:

*    An intuitive interface—Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
*    Easier debugging—Call ops directly to inspect running models and test changes. Use standard Python debugging tools for immediate error reporting.
*    Natural control flow—Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.

Eager execution supports most TensorFlow operations and GPU acceleration.

\\

Enabling eager execution changes how TensorFlow operations behave — now they immediately evaluate and return their values to Python. tf.Tensor objects reference concrete values instead of symbolic handles to nodes in a computational graph. **Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using print() or a debugger. Evaluating, printing, and checking tensor values does not break the flow for computing gradients.**

Eager execution works nicely with NumPy. NumPy operations accept tf.Tensor arguments. TensorFlow math operations convert Python objects and NumPy arrays to tf.Tensor objects. The tf.Tensor.numpy method returns the object's value as a NumPy ndarray.

**Setup**

In [0]:
tf.enable_eager_execution()

**Check if using**

In [0]:
tf.executing_eagerly() 

**Examples**

In [0]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m)) # No need of using tf.Session!!

In [0]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

In [0]:
# Broadcasting support
b = tf.add(a, 1)
print(b)

In [0]:
# Operator overloading is supported
print(a * b)

In [0]:
# Use NumPy values!!
c = np.multiply(a, b)
print(c)

In [0]:
# Obtain numpy value from a tensor:
print(a.numpy())

**You can even write a tf FizzBuzz**

In [0]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    for num in range(1, max_num.numpy()+1):
        num = tf.constant(num)
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num.numpy())
        counter += 1
        
fizzbuzz(24)

**Computing gradients**

During eager execution, use [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) to trace operations for computing gradients later.

In [0]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad)

Trainable variables are automatically watched. Tensors can be manually watched by invoking the **watch** method on this context manager.

In [0]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
    
dy_dx = g.gradient(y, x)
print(dy_dx)

GradientTapes can be nested to compute higher-order derivatives. For example,

In [0]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    with tf.GradientTape() as gg:
        gg.watch(x)
        y = x * x
    
    dy_dx = gg.gradient(y, x)
d2y_dx2 = g.gradient(dy_dx, x)

print(dy_dx)
print(d2y_dx2)

# Performance

Some models may experience increased overhead with eager execution enabled. Performance improvements are ongoing.

For compute-heavy models, such as ResNet50 training on a GPU, eager execution performance is comparable to graph execution. But this gap grows larger for models with less computation and there is work to be done for optimizing hot code paths for models with lots of small operations.

While eager execution makes development and debugging more interactive, TensorFlow graph execution has advantages for distributed training, performance optimizations, and production deployment. However, **writing graph code can feel different than writing regular Python code and more difficult to debug.**